In [ ]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline

In [ ]:
np.random.seed(42)

# Portfolio Planner

In this activity, you will use the Alpaca api to grab historical data for a 60/40 portfolio using `SPY` to represent the stock portion and `AGG` to represent the bonds.

In [ ]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# api = tradeapi.REST()
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

# Data Collection

In this step, you will need to use the Alpaca api to fetch closing prices for the `SPY` and `AGG` tickers. Save the results as a pandas DataFrame

In [ ]:
list_of_tickers = ["SPY", "AGG"]


In [ ]:
# Get all Asstes
assets = api.list_assets()

# Keep only tradeable assets
tradeable = [asset for asset in assets if asset.tradable ]
tradeable

In [ ]:
ticker = "SPY"

# Set timeframe to '1D'
timeframe = '1D'

# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-365)

# Get 1 year's worth of historical data for AAPL
spy_df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

spy_df.head()

In [ ]:
print(start_date)
print(end_date)

In [ ]:
# Drop Outer Table Level
spy_df = spy_df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
spy_df.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)

# Since this is daily data, we can keep only the date (remove the time) component of the data
spy_df.index = spy_df.index.date

spy_df.head()

In [ ]:
spy_df.columns = ['SPY close']

In [ ]:
ticker = "AGG"

# Set timeframe to '1D'
timeframe = '1D'

# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-365)

# Get 1 year's worth of historical data for AAPL
agg_df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

agg_df.head()

In [ ]:
# Drop Outer Table Level
agg_df = agg_df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
agg_df.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)


# Since this is daily data, we can keep only the date (remove the time) component of the data
agg_df.index = agg_df.index.date

agg_df

In [ ]:
agg_df.columns = ['Agg close']

In [ ]:
combined_port = pd.DataFrame()

In [ ]:
combined_port = pd.concat([spy_df, agg_df], axis="columns", join="inner")
combined_port.head()

# Monte Carlo Simulation

In this step, you will run Monte Carlo Simulations for your portfolio to model portfolio performance at different retirement ages. 

Complete the following steps:
1. Calculate the daily returns for the SPY and AGG closing prices.
2. Calculate volatility for both the SPY and AGG closing prices.
3. Find the last day's closing price for both stocks and save those as variables.
4. Run a Monte Carlo Simulation of at least 500 iterations and generate at least 30 years of closing prices

### HINTS:
There are 252 trading days per year, so the number of records to generate for each Monte Carlo run will be 252 days * 30 years

In [ ]:
# Use the `pct_change` function to calculate daily returns of SPY
daily_returns = combined_port.pct_change()
daily_returns.dropna
daily_returns

In [ ]:
# Calculate volatility
# Use the `mean` function to calculate the mean of daily returns for SPY
avg_daily_return_spy = daily_returns.mean()["SPY close"]
avg_daily_return_spy
std_daily_return_spy = daily_returns.std()['SPY close']
std_daily_return_spy

In [ ]:
avg_daily_return_agg = daily_returns.mean()["Agg close"]
avg_daily_return_agg
std_daily_return_agg = daily_returns.std()['Agg close']
std_daily_return_agg

In [ ]:
# Save the last day's closing price


In [ ]:
#number of simulations and trading days
num_simulations = 500
num_trading_days = 252 * 30

# last closing prices of AGG and AGG
SPY_last_price = combined_port['SPY close'][-1]
AGG_last_price = combined_port['Agg close'][-1]

# empty DataFrame to hold simulated prices for each simulation
simulated_price_df = pd.DataFrame()
portfolio_cumulative_returns = pd.DataFrame()

# Run the simulation of projecting stock prices for the next trading year, 500 times
for n in range(num_simulations):

    # Initialize the simulated prices list with the last closing price of SPY and AGG
    simulated_SPY_prices = [SPY_last_price]
    simulated_AGG_prices = [AGG_last_price]
    
    # Simulate the returns for 252 * 30 years
    for i in range(num_trading_days):
        
        # Calculate the simulated price using the last price within the list
        simulated_SPY_price = simulated_SPY_prices[-1] * (1 + np.random.normal(avg_daily_return_spy, std_daily_return_spy))
        simulated_AGG_price = simulated_AGG_prices[-1] * (1 + np.random.normal(avg_daily_return_agg, std_daily_return_agg))
        
        # Append the simulated price to the list
        simulated_SPY_prices.append(simulated_SPY_price)
        simulated_AGG_prices.append(simulated_AGG_price)
    
    # Append a simulated prices of each simulation to DataFrame
    simulated_price_df["SPY prices"] = pd.Series(simulated_SPY_prices)
    simulated_price_df["AGG prices"] = pd.Series(simulated_AGG_prices)
    
    # Calculate the daily returns of simulated prices
    simulated_daily_returns = simulated_price_df.pct_change()
    
    # Set the portfolio weights (60% SPY; 40% AGG)
    weights = [0.6, 0.4]

    # multiply weights with each column's simulated daily returns
    portfolio_daily_returns = simulated_daily_returns.dot(weights)
    
    # Calculate the normalized, cumulative return series
    portfolio_cumulative_returns[n] = (1 + portfolio_daily_returns.fillna(0)).cumprod()

# Print records from the DataFrame
# portfolio_cumulative_returns.head()

In [ ]:
# Visualize the Simulation

plot_title = "Simulations of portfolio Stock Trajectory"
portfolio_cumulative_returns.plot(legend=None, title=plot_title)

In [ ]:
# Select the last row for the cumulative returns (cumulative returns at 30 years)
ending_cumulative_returns = portfolio_cumulative_returns.iloc[-1, :]
ending_cumulative_returns

In [ ]:
# Display the 90% confidence interval for the ending returns
confidence_interval = ending_cumulative_returns.quantile(q=[0.05, 0.95])
confidence_interval

In [ ]:
# Visualize the distribution of the ending returns
ending_cumulative_returns.plot(kind='hist', bins=10)

---

# Retirement Analysis

In this section, you will use the monte carlo model to answer the following retirement planning questions:

1. What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?
2. Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?
3. Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?
4. How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

### What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?

In [ ]:
# 50th percentile

ending_cumulative_returns.median()

### Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?

In [ ]:
# # Set initial investment
initial_investment = 20000

# Calculate cumulative profit/loss of lower and upper bound
cumulative_pnl_lower_bound = initial_investment * ((confidence_interval.iloc[0] - (SPY_last_price + AGG_last_price)) / (SPY_last_price + AGG_last_price))
cumulative_pnl_upper_bound = initial_investment * ((confidence_interval.iloc[1] - (SPY_last_price + AGG_last_price)) / (SPY_last_price + AGG_last_price))

# Add the initial investment to the cumulative profit/loss of lower and upper bound
ending_investment_lower_bound = initial_investment + cumulative_pnl_lower_bound
ending_investment_upper_bound = initial_investment + cumulative_pnl_upper_bound

# Print the results
print(f"There is a 90% chance that an initial investment of $20,000 in SPY and AGG"
      f" over the next 252 trading days will end within in the range of"
      f" ${ending_investment_lower_bound} and ${ending_investment_upper_bound}")

In [ ]:
plt.figure()
ending_cumulative_returns.plot(kind='hist', density=True, bins=[0,1,2,3,4,5], title="Confidence interval")
plt.axvline(confidence_interval.iloc[0], color='r')
plt.axvline(confidence_interval.iloc[1], color='r')

### Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?

Note: This is effectively saying that 90% of the expected returns will be greater than the return at the 10th percentile, so this can help measure the uncertainty about having enough funds at retirement

In [ ]:
#Annual income = 6085$

.04 * (initial_investment * ending_cumulative_returns.quantile(.05))

### How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

In [ ]:
new_investment = (initial_investment* .5)
.04 * (new_investment * ending_cumulative_returns.quantile(.05))

### Optional Challenge

In this section, you will calculate and plot the cumulative returns for the median and 90% confidence intervals. This plot shows the expected cumulative returns for any given day between the first day and the last day of investment. 

In [ ]:
# YOUR CODE HERE